In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,roc_curve
from functions import decile_analysis
print('********* Required Libraries Imported *********')

********* Required Libraries Imported *********


In [2]:
#load model developing data
model_data = pd.read_parquet('Leobrix_Release_BBILAP_Combined_Data.parquet.gzip')
print('********* Data loaded ********* ')
print(model_data.shape)
# Make sure no duplicates in the data
model_data =  model_data.drop_duplicates(subset=['unique_reference_number'])
model_data =  model_data.drop_duplicates(subset=['latest_feature_id'])
print(model_data.shape)

********* Data loaded ********* 
(478328, 2831)
(478328, 2831)


In [3]:
#Create New Target (Good in 60-15 but bad in 90-24)
model_data['G15_B24'] = model_data.apply(
   lambda row: 0 if (row['60_in_15mob'] == 0 and row['90_in_24mob'] == 0)
    else ( 1 if (row['60_in_15mob'] == 0 and row['90_in_24mob'] == 1)
    else 2),axis=1)

model_data= model_data[model_data['G15_B24']!=2]
print('********* New Target Prepared *********')

********* New Target Prepared *********


In [4]:
model_data['G15_B24'].value_counts(dropna=False)

G15_B24
0    44460
1     3172
Name: count, dtype: int64

In [5]:
# take one copy for model development data
data = model_data.copy()
#some bank data have inf very rate but by safe replace 
data.replace([np.inf, -np.inf], np.nan, inplace=True)

In [6]:
#Split the time interval for model building and Validation of oot
model_df1 = data[(data['month_year']>='2021-01') & (data['month_year']<='2022-06')]
oot_df1= data[(data['month_year']>='2022-07') & (data['month_year']<='2022-09')]
print('********* Data Splitted *********')
print(model_df1.shape, oot_df1.shape)

********* Data Splitted *********
(34344, 2832) (10802, 2832)


In [7]:
 features_list = [
'TIME_SINCE_FIRST(raw.created_timestamp WHERE hour_part2 = 1)#ever',
'NUM_UNIQUE(pg_success_raw_data.YEAR(created_time_ist))',
'days_since_first_premium_bank_trans',
'NUM_UNIQUE(pg_success_raw_data.bank_title)',
'MEAN(raw.amount WHERE mode_cleaned = 2)#ever',
'max_one_time_spent',
'AVG_TIME_BETWEEN(raw.created_timestamp WHERE mode_cleaned = 2)#ever',
'distinct_merchant',
'days_since_first_ecom_trans',
'days_on_pg',

'month_year',
'G15_B24']

In [8]:

model_df = model_df1[features_list]
oot_df = oot_df1[features_list]

# Separate features (X) and target (y) for Model and OOT datasets
X_model = model_df.drop(columns=['month_year', 'G15_B24']) # Adjust 'target' to your actual target column name
y_model = model_df['G15_B24']

X_oot = oot_df.drop(columns=['month_year', 'G15_B24'])
y_oot = oot_df['G15_B24']

#Handling Null Values
for i in X_model.columns:
    X_model[i].fillna(-9999,inplace=True)

for i in X_oot.columns:
    X_oot[i].fillna(-9999,inplace=True)



/tmp/ipykernel_3041892/2552150070.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_model[i].fillna(-9999,inplace=True)
/tmp/ipykernel_3041892/2552150070.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try usi

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X_model, y_model, test_size=0.40,random_state=42)

In [10]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((20606, 10), (13738, 10), (20606,), (13738,))

In [11]:
#Split Training data into 80% train and 20% eval
X_train_new, X_eval, y_train_new, y_eval = train_test_split(
    X_train, y_train, test_size=0.2, random_state =42
)
print(X_train_new.shape, X_eval.shape, X_test.shape)

(16484, 10) (4122, 10) (13738, 10)


In [12]:
# 2. **Prepare DMatrix for XGBoost**
dtrain = xgb.DMatrix(X_train_new, label=y_train_new)
deval = xgb.DMatrix(X_eval, label=y_eval)
dtest = xgb.DMatrix(X_test, label=y_test)
doot = xgb.DMatrix(X_oot, label=y_oot)


In [13]:
# pg
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'learning_rate': 0.1,
     'max_depth': 2,
    'subsample': 0.3,
    'colsample_bytree': 0.3,
    'lambda': 8,
    'alpha': 5.8,
    'gamma': 6.4,
    'scale_pos_weight': 1,
    'random_state': 24,
}
evals = [(dtrain, 'train'), (deval, 'eval')]
xgb_model = xgb.train(
    params,
    dtrain,
    num_boost_round= 80,
    early_stopping_rounds= 40,
    evals=evals,
    verbose_eval=10
)

X_train_pred = X_train_new.copy()
X_eval_pred = X_eval.copy()
X_test_pred = X_test.copy()
X_oot_pred = X_oot.copy()


X_train_pred['y_actual'] = y_train_new
X_train_pred['y_pred'] = xgb_model.predict(dtrain)
X_train_pred['LID'] = model_df1.loc[X_train_new.index,'latest_feature_id'].values

X_eval_pred['y_actual'] = y_eval
X_eval_pred['y_pred'] = xgb_model.predict(deval)
X_eval_pred['LID'] = model_df1.loc[X_eval.index,'latest_feature_id'].values

X_test_pred['y_actual'] = y_test
X_test_pred['y_pred'] = xgb_model.predict(dtest)
X_test_pred['LID'] = model_df1.loc[X_test.index,'latest_feature_id'].values

X_oot_pred['y_actual'] = y_oot
X_oot_pred['y_pred'] = xgb_model.predict(doot)
X_oot_pred['LID'] = oot_df1.loc[X_oot.index,'latest_feature_id'].values

[0]	train-auc:0.50000	eval-auc:0.50000
[10]	train-auc:0.55623	eval-auc:0.59871
[20]	train-auc:0.56016	eval-auc:0.58999
[30]	train-auc:0.56127	eval-auc:0.60344
[40]	train-auc:0.56406	eval-auc:0.60437
[50]	train-auc:0.56371	eval-auc:0.60569
[60]	train-auc:0.56371	eval-auc:0.60569
[70]	train-auc:0.56352	eval-auc:0.60561
[79]	train-auc:0.56341	eval-auc:0.60552


In [14]:
# 6. **Run Decile Analysis**
train_summary, train_auc, train_gini, train_ks = decile_analysis(X_train_pred, 'y_pred', 'y_actual')
eval_summary, eval_auc, eval_gini, eval_ks = decile_analysis(X_eval_pred, 'y_pred', 'y_actual')

test_summary, test_auc, test_gini, test_ks = decile_analysis(X_test_pred, 'y_pred', 'y_actual')
oot_summary, oot_auc, oot_gini, oot_ks = decile_analysis(X_oot_pred, 'y_pred', 'y_actual')

# 7. **Print Results**
print(f"Train AUC: {train_auc:.2f}, Gini: {train_gini:.2f}, KS: {train_ks:.2f}")
print(f"Eval AUC: {eval_auc:.2f}, Gini: {eval_gini:.2f}, KS: {eval_ks:.2f}")
print(f"Test AUC: {test_auc:.2f}, Gini: {test_gini:.2f}, KS: {test_ks:.2f}")
print(f"OOT AUC: {oot_auc:.2f}, Gini: {oot_gini:.2f}, KS: {oot_ks:.2f}")


Train AUC: 0.56, Gini: 0.13, KS: 0.11
Eval AUC: 0.61, Gini: 0.21, KS: 0.18
Test AUC: 0.56, Gini: 0.13, KS: 0.11
OOT AUC: 0.55, Gini: 0.11, KS: 0.10


In [15]:
oot_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
7          0.090824         0.080094        0.080674        0.075154   3204   
6          0.079012         0.069598        0.073672        0.064814    810   
5          0.085546         0.063211        0.064521        0.062122   1356   
4          0.085422         0.060615        0.061309        0.059366   1077   
3          0.068493         0.058862        0.059321        0.058619   1022   
2          0.065068         0.057576        0.058601        0.057146    584   
1          0.054929         0.055860        0.057103        0.055285   2749   

        label_good  label_bad  
Decile                         
7             2913        291  
6              746         64  
5             1240        116  
4              985         92  
3              952         70  
2              546         38  
1             2598        151

In [16]:
train_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
7          0.086744         0.080114        0.080674        0.075154   4911   
6          0.064303         0.067936        0.073672        0.064521   1664   
5          0.054667         0.062488        0.064452        0.060938   1500   
4          0.064231         0.060326        0.060892        0.059321   1697   
3          0.053763         0.058673        0.059021        0.057437   1674   
2          0.065491         0.057123        0.057146        0.057103   1588   
1          0.046667         0.055455        0.056756        0.055285   3450   

        label_good  label_bad  
Decile                         
7             4485        426  
6             1557        107  
5             1418         82  
4             1588        109  
3             1584         90  
2             1484        104  
1             3289        161

In [17]:
test_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
7          0.078979         0.079606        0.080674        0.070848   4115   
6          0.068130         0.066591        0.070842        0.064160   1321   
5          0.059034         0.062231        0.062929        0.060938   1118   
4          0.055398         0.060325        0.060892        0.059321   1408   
3          0.052671         0.058675        0.059021        0.057437   1348   
2          0.044444         0.057123        0.057146        0.057103   1395   
1          0.049456         0.055443        0.056756        0.055285   3033   

        label_good  label_bad  
Decile                         
7             3790        325  
6             1231         90  
5             1052         66  
4             1330         78  
3             1277         71  
2             1333         62  
1             2883        150

In [18]:
feature_importance = xgb_model.get_score(importance_type='weight')
feature_importance_df = pd.DataFrame(
    list(feature_importance.items()),
    columns =['Feature','Importance']
).sort_values(by='Importance',ascending=False)
print(feature_importance_df)

                                             Feature  Importance
0  TIME_SINCE_FIRST(raw.created_timestamp WHERE h...         5.0
4                                         days_on_pg         4.0
1                days_since_first_premium_bank_trans         1.0
2       MEAN(raw.amount WHERE mode_cleaned = 2)#ever         1.0
3                                 max_one_time_spent         1.0


In [19]:
feature_importance_df

Feature  Importance
0  TIME_SINCE_FIRST(raw.created_timestamp WHERE h...         5.0
4                                         days_on_pg         4.0
1                days_since_first_premium_bank_trans         1.0
2       MEAN(raw.amount WHERE mode_cleaned = 2)#ever         1.0
3                                 max_one_time_spent         1.0

In [20]:
#Save the Pred
Train = X_train_pred[['LID','y_actual','y_pred']]
Eval = X_eval_pred[['LID','y_actual','y_pred']]
Test = X_test_pred[['LID','y_actual','y_pred']]
Oot = X_oot_pred[['LID','y_actual','y_pred']]
Train.to_csv('LeoPayu_PG_Train_Pred_1524.csv',index=False)
Eval.to_csv('LeoPayu_PG_Eval_Pred_1524.csv',index=False)
Test.to_csv('LeoPayu_PG_Test_Pred_1524.csv',index=False)
Oot.to_csv('LeoPayu_PG_Oot_Pred_1524.csv',index=False)


In [21]:
#Save the model
xgb_model.save_model('LeoPayu_pg_15_in_24mob.json')
print('Model Saved')

Model Saved
